# Coursera Case Study 1: Revisited

## Introduction
This notebook is a repeat of the work done on a case study for the Coursera's Google Data Analytics Certificate program. The work was previously performed in Excel and R, with an RMarkdown HTML document as the output. The goal here is to repeat and document the steps taken to clean and analyze the data. This is sorely missing from the write-up found over at my [Kaggle site](https://www.kaggle.com/code/scwilso28/coursera-case-study-1-cyclistic-data-analysis).

The data cleanup steps performed in my other Jupyter notebook on [Github](https://github.com/cwilso28/my-first-jupyter-notebook) will be used here, but with some modifications to account for the large datafiles used in the case study.

## Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import zipfile
from os import listdir

In [2]:
to_import = ['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'end_station_name', 'member_casual']
my_files = [f for f in listdir('Datasets')]

In [3]:
zf = zipfile.ZipFile('Datasets/'+my_files[0])
df = pd.read_csv(zf.open(zf.namelist()[0]), usecols = to_import)


In [4]:
for file in my_files[1:]:
    zf = zipfile.ZipFile('Datasets/' + file)
    data = pd.read_csv(zf.open(zf.namelist()[0]), usecols = to_import)
    print('File name: ' + zf.namelist()[0] + '; File length: ' + str(len(data)))
    df = pd.concat([df, data])


File name: 202104-divvy-tripdata.csv; File length: 337230
File name: 202105-divvy-tripdata.csv; File length: 531633
File name: 202106-divvy-tripdata.csv; File length: 729595
File name: 202107-divvy-tripdata.csv; File length: 822410
File name: 202108-divvy-tripdata.csv; File length: 804352
File name: 202109-divvy-tripdata.csv; File length: 756147
File name: 202110-divvy-tripdata.csv; File length: 631226
File name: 202111-divvy-tripdata.csv; File length: 359978
File name: 202112-divvy-tripdata.csv; File length: 247540
File name: 202201-divvy-tripdata.csv; File length: 103770
File name: 202202-divvy-tripdata.csv; File length: 115609
File name: 202203-divvy-tripdata.csv; File length: 284042


In [5]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual
0,CFA86D4455AA1030,classic_bike,2021-03-16 08:32:30,2021-03-16 08:36:34,Humboldt Blvd & Armitage Ave,Stave St & Armitage Ave,casual
1,30D9DC61227D1AF3,classic_bike,2021-03-28 01:26:28,2021-03-28 01:36:55,Humboldt Blvd & Armitage Ave,Central Park Ave & Bloomingdale Ave,casual
2,846D87A15682A284,classic_bike,2021-03-11 21:17:29,2021-03-11 21:33:53,Shields Ave & 28th Pl,Halsted St & 35th St,casual
3,994D05AA75A168F2,classic_bike,2021-03-11 13:26:42,2021-03-11 13:55:41,Winthrop Ave & Lawrence Ave,Broadway & Sheridan Rd,casual
4,DF7464FBE92D8308,classic_bike,2021-03-21 09:09:37,2021-03-21 09:27:33,Glenwood Ave & Touhy Ave,Chicago Ave & Sheridan Rd,casual


In [6]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5952028 entries, 0 to 284041
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   ride_id             object
 1   rideable_type       object
 2   started_at          object
 3   ended_at            object
 4   start_station_name  object
 5   end_station_name    object
 6   member_casual       object
dtypes: object(7)
memory usage: 2.8 GB


In [7]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['member_casual'] = df['member_casual'].map({'casual': 0, 'member': 1})

In [8]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5952028 entries, 0 to 284041
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   end_station_name    object        
 6   member_casual       int64         
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 1.8 GB


In [9]:
duplicate_check = ['rideable_type', 'started_at', 'ended_at', 'start_station_name', 'end_station_name', 'member_casual']
duplicate_entries = df[df.duplicated(subset = duplicate_check, keep = False)]
duplicate_entries = duplicate_entries.drop_duplicates(subset = duplicate_check)
print(len(duplicate_entries))

565


In [10]:
print(100*len(duplicate_entries)/len(df))

0.009492562871008


In [11]:
print(len(df))

5952028


In [12]:
df.drop_duplicates(subset = duplicate_check, inplace = True)
print(len(df))

5951462


In [13]:
df['ride_duration'] = df.ended_at - df.started_at
df[['started_at', 'ended_at', 'ride_duration']].head()

,started_at,ended_at,ride_duration
0,2021-03-16 08:32:30,2021-03-16 08:36:34,0 days 00:04:04
1,2021-03-28 01:26:28,2021-03-28 01:36:55,0 days 00:10:27
2,2021-03-11 21:17:29,2021-03-11 21:33:53,0 days 00:16:24
3,2021-03-11 13:26:42,2021-03-11 13:55:41,0 days 00:28:59
4,2021-03-21 09:09:37,2021-03-21 09:27:33,0 days 00:17:56


In [14]:
print(df.ride_duration.min())

-1 days +23:01:58


In [15]:
zero_timedelta = pd.to_timedelta(0)
column = df['ride_duration']
print(column[column < zero_timedelta].count())

147


In [16]:
# df['ride_duration'].to_numpy()[df['ride_duration'].to_numpy() < zero_timedelta] = zero_timedelta
df.loc[df['ride_duration'] < zero_timedelta] = zero_timedelta
print(column[column < zero_timedelta].count())

0


In [17]:
df['ride_duration_hrs'] = df.ride_duration / np.timedelta64(1, 'h')
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_duration,ride_duration_hrs
0,CFA86D4455AA1030,classic_bike,2021-03-16 08:32:30,2021-03-16 08:36:34,Humboldt Blvd & Armitage Ave,Stave St & Armitage Ave,0,0 days 00:04:04,0.067778
1,30D9DC61227D1AF3,classic_bike,2021-03-28 01:26:28,2021-03-28 01:36:55,Humboldt Blvd & Armitage Ave,Central Park Ave & Bloomingdale Ave,0,0 days 00:10:27,0.174167
2,846D87A15682A284,classic_bike,2021-03-11 21:17:29,2021-03-11 21:33:53,Shields Ave & 28th Pl,Halsted St & 35th St,0,0 days 00:16:24,0.273333
3,994D05AA75A168F2,classic_bike,2021-03-11 13:26:42,2021-03-11 13:55:41,Winthrop Ave & Lawrence Ave,Broadway & Sheridan Rd,0,0 days 00:28:59,0.483056
4,DF7464FBE92D8308,classic_bike,2021-03-21 09:09:37,2021-03-21 09:27:33,Glenwood Ave & Touhy Ave,Chicago Ave & Sheridan Rd,0,0 days 00:17:56,0.298889


## Data Analysis